In [1]:
import os

In [2]:
import json

def parse_configuration(config_file):
    """Loads config file if a string was passed
        and returns the input if a dictionary was passed.
    """
    if isinstance(config_file, str):
        with open(config_file, 'r') as json_file:
            return json.load(json_file)
    else:
        return config_file

In [3]:
path_split_aug_summe = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/summe_aug_splits.json"
splits_aug_summe = parse_configuration(path_split_aug_summe)

In [8]:
import torch 
import h5py

class VSMDataset(torch.utils.data.Dataset):
    """Video Summarizer Dataset
        Datasets: TVSum, Summe, VSUMM, CoSum, Visiocity
    """

    def __init__(self, hdfs_path, split=None, key_split=None,
                 googlenet=False, 
                 resnext=False, 
                 inceptionv3=False,
                 i3d_rgb=False,
                 i3d_flow=False,
                 resnet3d=False
                ):
        """
        Args:
           hdfs_path (string): path of the hdfs processed data
           split (dict): idxs of the train/test split 
        """
        if not isinstance(hdfs_path, list):
            hdfs_paths = [hdfs_path]
        else:
            hdfs_paths = hdfs_path 
        
        self.labels = {}
        self.data = {}

        keys_to_avoid = ['gtscore', 'gtsummary', 'user_summary']
        
        if not googlenet:
            keys_to_avoid.append('features')
        if not resnext:
            keys_to_avoid.append('features_rn')
        if not inceptionv3:
            keys_to_avoid.append('features_iv3')
        if not i3d_rgb:
            keys_to_avoid.append('features_rgb')
        if not i3d_flow:
            keys_to_avoid.append('features_flow')
        if not resnet3d:
            keys_to_avoid.append('features_3D')        
        
        iterator_videos = 0
        for path in hdfs_paths:
            videos_info = h5py.File(path)  

            if 'tvsum' in path:
                name_dataset = 'tvsum'
            elif 'summe' in path:
                name_dataset = 'summe'
            elif 'ovp' in path:
                name_dataset = 'ovp'
            elif 'youtube' in path:
                name_dataset = 'youtube'
            elif 'cosum' in path:
                name_dataset = 'cosum'
            elif 'mvs1k' in path:
                name_dataset = 'mvs1k'
            elif 'visiocity' in path:
                name_dataset = 'visiocity'
            
            videos_to_iterate = list(videos_info)
            if split:
                if len(hdfs_paths)==1:
                    videos_to_iterate = [video for video in videos_to_iterate if video in split[key_split]]
                else:
                    key_videos = [video.split('/')[-1] for video in split[key_split] if name_dataset in video]
                    videos_to_iterate = [video for video in videos_to_iterate if video in key_videos]
                    

            for it, video in enumerate(videos_to_iterate):
                self.labels[iterator_videos] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key in ('gtscore', 'gtsummary', 'user_summary') )
                self.data[iterator_videos] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key not in keys_to_avoid )   
                self.data[iterator_videos]['name_dataset'] = name_dataset
                if "video_name" in self.data[it].keys():
                    self.data[iterator_videos]["video_name"] = str(self.data[iterator_videos]["video_name"]) 
                iterator_videos = iterator_videos + 1
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]        
        return X, y
    
    def get_feature(self, index, feature):
        X = self.data[index][feature]
        return X

def show_sample(idx, hdfs_path):

    vsm_dataset = VSMDataset(hdfs_path=hdfs_path)
    video_info, label = vsm_dataset[idx]

    return video_info, label

In [9]:
params = {'batch_size': 1,
          'num_workers': 2}
max_epochs = 100

In [10]:
paths = [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        ]

In [11]:
for path in paths:

    training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                             i3d_rgb=True, i3d_flow=True, resnet3d=True)

    training_generator = torch.utils.data.DataLoader(training_set, **params)
    it = 0
    for video_info, label in training_generator:
        it = it + 1
        continue
    print("done", path)
    print(it)
        
    #break



done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
50
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
25
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5
50
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5
39
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5
41
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
50
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
25


In [57]:
video_info["name_dataset"]

['summe']

In [12]:
path_split_aug_summe = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/summe_aug_splits.json"
splits_aug_summe = parse_configuration(path_split_aug_summe)

itsplit = 0
for split in splits_aug_summe:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    array_paths = [
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
            ]]
    
    for paths in array_paths:
    
        training_set = VSMDataset(paths, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params, shuffle=True)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done")
        print(it)
        #break
    print()
    #break

new split-> 1
done
159
done
159
done
159
done
159

new split-> 2
done
159
done
159
done
159
done
159

new split-> 3
done
159
done
159
done
159
done
159

new split-> 4
done
159
done
159
done
159
done
159

new split-> 5
done
159
done
159
done
159
done
159



In [66]:
video_info["name_dataset"]

['ovp']

In [61]:
video_info

{'change_points': tensor([[[   0,  239],
          [ 240,  299],
          [ 300,  389],
          [ 390,  539],
          [ 540,  629],
          [ 630,  764],
          [ 765,  974],
          [ 975, 1094],
          [1095, 1199],
          [1200, 1274],
          [1275, 1424],
          [1425, 1769],
          [1770, 2099],
          [2100, 2174],
          [2175, 2264],
          [2265, 2399],
          [2400, 2609],
          [2610, 2683]]]),
 'features': tensor([[[8.2394e-05, 7.9756e-05, 1.2095e-03,  ..., 5.6084e-03,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 1.0901e-03, 0.0000e+00,  ..., 9.4123e-04,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 2.7174e-04, 0.0000e+00,  ..., 1.0270e-03,
           0.0000e+00, 0.0000e+00],
          ...,
          [3.7223e-02, 8.0595e-02, 7.1901e-03,  ..., 9.9105e-02,
           0.0000e+00, 1.8123e-04],
          [2.2474e-02, 4.5581e-02, 2.4407e-02,  ..., 2.4914e-02,
           0.0000e+00, 0.0000e+00],
          [2.780

In [28]:
path_split_canonical_summe = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/summe_splits.json"
splits_canonical_summe = parse_configuration(path_split_canonical_summe)

itsplit = 0
for split in splits_canonical_summe:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    paths = [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        ]
    for path in paths:
    
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done", path)
        print(it)
        #break
    print()
    #break

new split-> 1
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 2
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 3
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 4
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
don

In [21]:
training_set[0][0]["features_rn"].shape

(707, 2048)

In [15]:
video_info['features_rn'].shape

torch.Size([1, 468, 2048])

In [16]:
path_split_canonical_summe_no = "/home/shuaman/video_sm/video_summarization/splits/msva_splits/summe_ordered_non_overlap_splits.json"
splits_canonical_summe_no = parse_configuration(path_split_canonical_summe_no)

itsplit = 0
for split in splits_canonical_summe_no:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    paths = [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        ]
    for path in paths:
    
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done", path)
        print(it)
        #break
    print()
    #break

new split-> 1
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 2
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
20
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5
20

new split-> 3
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
done /data/shuaman/video_summarization/datasets/

In [13]:
path_split_canonical_tvsum = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/tvsum_splits.json"
splits_canonical_tvsum = parse_configuration(path_split_canonical_tvsum)

itsplit = 0
for split in splits_canonical_tvsum:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    paths = [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        ]
    for path in paths:
    
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done", path)
        print(it)
        #break
    print()
    #break

new split-> 1
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 2
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 3
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 4
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
don

In [21]:
path_split_canonical_tvsum_no = "/home/shuaman/video_sm/video_summarization/splits/msva_splits/tvsum_random_non_overlap_splits.json"
splits_canonical_tvsum_no = parse_configuration(path_split_canonical_tvsum_no)

itsplit = 0
for split in splits_canonical_tvsum_no:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    paths = [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        ]
    for path in paths:
    
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done", path)
        print(it)
        #break
    print()
    #break

new split-> 1
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 2
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
40
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
done /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
40

new split-> 3
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
done /data/shuaman/video_summarization/datasets/

In [18]:
path_split_aug_tvsum = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/tvsum_aug_splits.json"
splits_aug_tvsum = parse_configuration(path_split_aug_tvsum)

itsplit = 0
for split in splits_aug_tvsum:
    itsplit = itsplit + 1
    print("new split->", itsplit)
    array_paths = [
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
            ],
            [
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
            "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
            ]]
    
    for paths in array_paths:
    
        training_set = VSMDataset(paths, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True, split=split, key_split="train_keys")

        training_generator = torch.utils.data.DataLoader(training_set, **params)
        #for epoch in range(max_epochs):
        it = 0
        for video_info, label in training_generator:
            it = it + 1
            continue
        print("done")
        print(it)
        #break
    print()
    #break

new split-> 1
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5
done
154
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/data

In [22]:
#transfer setting


array_paths = [
        [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        ],
        [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        ],
        [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        ],
        [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        ]]

for paths in array_paths:

    training_set = VSMDataset(paths, googlenet=True, resnext=True, inceptionv3=True,
                             i3d_rgb=True, i3d_flow=True, resnet3d=True)

    training_generator = torch.utils.data.DataLoader(training_set, **params)
    #for epoch in range(max_epochs):
    it = 0
    for video_info, label in training_generator:
        it = it + 1
        continue
    print("done")
    print(it)
    #break
print()
#break

path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5
done
205
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5
path /data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_proc

# Complete all the other features

In [5]:
import numpy as np

### TVSum

1. Nos quedamos con MSVA pues ambos son iguales

In [13]:
#
path_tvsum_msva = "/data/shuaman/video_summarization/datasets/object_features/eccv16_dataset_tvsum_google_pool5.h5"
path_tvsum_dsnet = "/data/shuaman/video_summarization/datasets/dsnet/datasets/eccv16_dataset_tvsum_google_pool5.h5"

In [14]:
data_tvsum_msva = h5py.File(path_tvsum_msva, 'r')
data_tvsum_dsnet = h5py.File(path_tvsum_dsnet, 'r')
#data_tvsum_dsnet = h5py.File("/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5", 'r')

In [165]:
data_tvsum_msva.close()
data_tvsum_dsnet.close()

In [200]:
#CONCLUSION AMBOS SON LO MISMO, SOLO SIRVE QUEDARNOS CON UNO: SERA MSVA PORQUE FALTA CRUZAR CON FLOW Y RGB
for i in range(0,50):
    video_gt = data_tvsum_msva["video_" + str(i+1)]
    video_mine = data_tvsum_dsnet["video_" + str(i+1)]
    
    perfect = True
    
    print(f'video {i+1} comparison')
    if video_gt['features'][...].shape != video_mine['features'][...].shape:
        perfect = False
        print("features gt", video_gt['features'][...].shape)
        print("features mine", video_mine['features'][...].shape)
        print()
    print(np.all(video_gt['features'][...]==video_mine['features'][...]))    
    
    if video_gt['gtscore'][...].shape != video_mine['gtscore'][...].shape:
        perfect = False
        print("gtscore gt", video_gt['gtscore'][...].shape)
        print("gtscore mine", video_mine['gtscore'][...].shape)
        print()
    print(np.all(video_gt['gtscore'][...]==video_mine['gtscore'][...])) 
#     print("video name gt", video_gt['video_name'][...])
#     print("video name mine", video_mine['video_name'][...])
    
    if video_gt['user_summary'][...].shape != video_mine['user_summary'][...].shape:
        perfect = False
        print("user summary gt", video_gt['user_summary'][...].shape)
        print("user summary mine", video_mine['user_summary'][...].shape)
        print()
    print(np.all(video_gt['user_summary'][...]==video_mine['user_summary'][...])) 
        
    if video_gt['n_frames'][...] != video_mine['n_frames'][...]:
        perfect = False
        print("n frames gt", video_gt['n_frames'][...])
        print("n frames mine", video_mine['n_frames'][...])
        print()
    if video_gt['change_points'][...].shape != video_mine['change_points'][...].shape:  
        perfect = False
        print("change points gt", video_gt['change_points'][...].shape)
        print("change points mine", video_mine['change_points'][...].shape)
        print()
    print(np.all(video_gt['change_points'][...]==video_mine['change_points'][...])) 
        
    if video_gt['n_frame_per_seg'][...].shape != video_mine['n_frame_per_seg'][...].shape: 
        perfect = False
        print("n frame per seg gt", video_gt['n_frame_per_seg'][...].shape)
        print("n frame per seg mine", video_mine['n_frame_per_seg'][...].shape)
        print()
    print(np.all(video_gt['n_frame_per_seg'][...]==video_mine['n_frame_per_seg'][...])) 
    
    if video_gt['picks'][...].shape != video_mine['picks'][...].shape:
        perfect = False
        print("picks gt", video_gt['picks'][...].shape)
        print("picks mine", video_mine['picks'][...].shape)
        print()
    print(np.all(video_gt['picks'][...]==video_mine['picks'][...])) 
        
    if video_gt['n_steps'][...] != video_mine['n_steps'][...]:
        perfect = False
        print("n steps gt", video_gt['n_steps'][...])
        print("n steps mine", video_mine['n_steps'][...])

        
    if perfect: 
        print("Everything is perfect")
        print()

video 1 comparison
True
True
True
True
True
True
Everything is perfect

video 2 comparison
True
True
True
True
True
True
Everything is perfect

video 3 comparison
True
True
True
True
True
True
Everything is perfect

video 4 comparison
True
True
True
True
True
True
Everything is perfect

video 5 comparison
True
True
True
True
True
True
Everything is perfect

video 6 comparison
True
True
True
True
True
True
Everything is perfect

video 7 comparison
True
True
True
True
True
True
Everything is perfect

video 8 comparison
True
True
True
True
True
True
Everything is perfect

video 9 comparison
True
True
True
True
True
True
Everything is perfect

video 10 comparison
True
True
True
True
True
True
Everything is perfect

video 11 comparison
True
True
True
True
True
True
Everything is perfect

video 12 comparison
True
True
True
True
True
True
Everything is perfect

video 13 comparison
True
True
True
True
True
True
Everything is perfect

video 14 comparison
True
True
True
True
True
True
Everything

In [8]:
# data_tvsum_msva.close()

In [11]:
# list(video_mine.keys())

In [88]:
import glob

In [105]:
path_feature_flow = "/data/shuaman/video_summarization/datasets/kinetic_features/tvsum/FLOW/features"
path_feature_rgb = "/data/shuaman/video_summarization/datasets/kinetic_features/tvsum/RGB/features"

In [171]:
path_tvsum_mine = "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5"
data_tvsum_mine = h5py.File(path_tvsum_mine, 'r')
#CONCLUSION AMBOS SON LO MISMO, SOLO SIRVE QUEDARNOS CON UNO: SERA MSVA PORQUE FALTA CRUZAR CON FLOW Y RGB
data_tvsum_msva = h5py.File(path_tvsum_msva, 'r+')
#data_tvsum_dsnet = h5py.File(path_tvsum_dsnet, 'r')
for i in range(0,50):
    #video_gt = data_tvsum_msva["video_" + str(i+1)]
    video_mine = data_tvsum_mine["video_" + str(i+1)]
    video_name = str(video_mine['video_name'][...])[2:-1]
    try:
    # add features
#         print(video_mine['features_rn'][...])
        data_tvsum_msva["video_" + str(i+1)]['features_rgb'] = np.load(os.path.join(path_feature_rgb,f'{video_name}.npy'))
        data_tvsum_msva["video_" + str(i+1)]['features_flow'] = np.load(os.path.join(path_feature_flow,f'{video_name}.npy'))
        data_tvsum_msva["video_" + str(i+1)]['video_name'] = data_tvsum_mine["video_" + str(i+1)]['video_name'][...]
        data_tvsum_msva["video_" + str(i+1)]['features_rn'] = data_tvsum_mine["video_" + str(i+1)]['features_rn'][...]
        data_tvsum_msva["video_" + str(i+1)]['features_iv3'] = data_tvsum_mine["video_" + str(i+1)]['features_iv3'][...]
        data_tvsum_msva["video_" + str(i+1)]['features_3D'] = data_tvsum_mine["video_" + str(i+1)]['features_3D'][...]
    except:
        try:
            data_tvsum_msva["video_" + str(i+1)]['features_rgb'][...] = np.load(os.path.join(path_feature_rgb,f'{video_name}.npy'))
            data_tvsum_msva["video_" + str(i+1)]['features_flow'][...] = np.load(os.path.join(path_feature_flow,f'{video_name}.npy'))
            data_tvsum_msva["video_" + str(i+1)]['video_name'][...] = data_tvsum_mine["video_" + str(i+1)]['video_name'][...]
            data_tvsum_msva["video_" + str(i+1)]['features_rn'][...] = data_tvsum_mine["video_" + str(i+1)]['features_rn'][...]
            data_tvsum_msva["video_" + str(i+1)]['features_iv3'][...] = data_tvsum_mine["video_" + str(i+1)]['features_iv3'][...]
            data_tvsum_msva["video_" + str(i+1)]['features_3D'][...] = data_tvsum_mine["video_" + str(i+1)]['features_3D'][...]
        except:
            print("Error")
    #print(sorted(list(video_mine.keys())) == sorted(list(data_tvsum_msva["video_" + str(i+1)].keys())))     
data_tvsum_msva.close()

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [170]:
data_tvsum_msva.close()

In [168]:
data_tvsum_msva = h5py.File(path_tvsum_msva, 'r')
video_gt = data_tvsum_msva["video_" + str(25)]

In [169]:
print(len(list(data_tvsum_msva.keys())))
print(len(list(video_gt.keys())))
list(video_gt.keys())

50
15


['change_points',
 'features',
 'features_3D',
 'features_flow',
 'features_iv3',
 'features_rgb',
 'features_rn',
 'gtscore',
 'gtsummary',
 'n_frame_per_seg',
 'n_frames',
 'n_steps',
 'picks',
 'user_summary',
 'video_name']

### SumMe
1. Nos quedamos con MSVA pues son iguales

In [6]:

path_summe_msva = "/data/shuaman/video_summarization/datasets/object_features/eccv16_dataset_summe_google_pool5.h5"
path_summe_dsnet = "/data/shuaman/video_summarization/datasets/dsnet/datasets/eccv16_dataset_summe_google_pool5.h5"

In [181]:
data_summe_msva = h5py.File(path_summe_msva, 'r')
data_summe_dsnet = h5py.File(path_summe_dsnet, 'r')

In [183]:
for i in range(0,25):
    video_gt = data_summe_msva["video_" + str(i+1)]
    video_mine = data_summe_dsnet["video_" + str(i+1)]
    
    perfect = True
    
    print(f'video {i+1} comparison')
    if video_gt['features'][...].shape != video_mine['features'][...].shape:
        perfect = False
        print("features gt", video_gt['features'][...].shape)
        print("features mine", video_mine['features'][...].shape)
        print()
    print(np.all(video_gt['features'][...]==video_mine['features'][...]))
        
    if video_gt['gtscore'][...].shape != video_mine['gtscore'][...].shape:
        perfect = False
        print("gtscore gt", video_gt['gtscore'][...].shape)
        print("gtscore mine", video_mine['gtscore'][...].shape)
        print()
    print(np.all(video_gt['gtscore'][...]==video_mine['gtscore'][...]))
        
    print("video name gt", video_gt['video_name'][...])
    print("video name mine", video_mine['video_name'][...])
    
    if video_gt['user_summary'][...].shape != video_mine['user_summary'][...].shape:
        perfect = False
        print("user summary gt", video_gt['user_summary'][...].shape)
        print("user summary mine", video_mine['user_summary'][...].shape)
        print()
    print(np.all(video_gt['user_summary'][...]==video_mine['user_summary'][...]))
        
    if video_gt['n_frames'][...] != video_mine['n_frames'][...]:
        perfect = False
        print("n frames gt", video_gt['n_frames'][...])
        print("n frames mine", video_mine['n_frames'][...])
        print()
    if video_gt['change_points'][...].shape != video_mine['change_points'][...].shape:  
        perfect = False
        print("change points gt", video_gt['change_points'][...].shape)
        print("change points mine", video_mine['change_points'][...].shape)
        print()
    print(np.all(video_gt['change_points'][...]==video_mine['change_points'][...]))
        
    if video_gt['n_frame_per_seg'][...].shape != video_mine['n_frame_per_seg'][...].shape: 
        perfect = False
        print("n frame per seg gt", video_gt['n_frame_per_seg'][...].shape)
        print("n frame per seg mine", video_mine['n_frame_per_seg'][...].shape)
        print()
    print(np.all(video_gt['n_frame_per_seg'][...]==video_mine['n_frame_per_seg'][...]))
        
    if video_gt['picks'][...].shape != video_mine['picks'][...].shape:
        perfect = False
        print("picks gt", video_gt['picks'][...].shape)
        print("picks mine", video_mine['picks'][...].shape)
        print()
    print(np.all(video_gt['picks'][...]==video_mine['picks'][...]))
        
    if video_gt['n_steps'][...] != video_mine['n_steps'][...]:
        perfect = False
        print("n steps gt", video_gt['n_steps'][...])
        print("n steps mine", video_mine['n_steps'][...])
        
    if perfect: 
        print("Everything is perfect")
        print()

video 1 comparison
True
True
video name gt b'Air_Force_One'
video name mine b'Air_Force_One'
True
True
True
True
Everything is perfect

video 2 comparison
True
True
video name gt b'Base jumping'
video name mine b'Base jumping'
True
True
True
True
Everything is perfect

video 3 comparison
True
True
video name gt b'Bearpark_climbing'
video name mine b'Bearpark_climbing'
True
True
True
True
Everything is perfect

video 4 comparison
True
True
video name gt b'Bike Polo'
video name mine b'Bike Polo'
True
True
True
True
Everything is perfect

video 5 comparison
True
True
video name gt b'Bus_in_Rock_Tunnel'
video name mine b'Bus_in_Rock_Tunnel'
True
True
True
True
Everything is perfect

video 6 comparison
True
True
video name gt b'Car_railcrossing'
video name mine b'Car_railcrossing'
True
True
True
True
Everything is perfect

video 7 comparison
True
True
video name gt b'Cockpit_Landing'
video name mine b'Cockpit_Landing'
True
True
True
True
Everything is perfect

video 8 comparison
True
True
v

In [7]:
path_feature_flow = "/data/shuaman/video_summarization/datasets/kinetic_features/summe/FLOW/features"
path_feature_rgb = "/data/shuaman/video_summarization/datasets/kinetic_features/summe/RGB/features"

In [160]:
path_summe_mine = "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5"
data_summe_mine = h5py.File(path_summe_mine, 'r')
#CONCLUSION AMBOS SON LO MISMO, SOLO SIRVE QUEDARNOS CON UNO: SERA MSVA PORQUE FALTA CRUZAR CON FLOW Y RGB
data_summe_msva = h5py.File(path_summe_msva, 'r+')
#data_tvsum_dsnet = h5py.File(path_tvsum_dsnet, 'r')
for i in range(0,25):
    #video_gt = data_tvsum_msva["video_" + str(i+1)]
    video_mine = data_summe_mine["video_" + str(i+1)]
    video_name = str(video_mine['video_name'][...])[2:-1]
    try:
    # add features
#         print(video_mine['features_rn'][...])
        data_summe_msva["video_" + str(i+1)]['features_rgb'] = np.load(os.path.join(path_feature_rgb,f'{video_name}.npy'))
        data_summe_msva["video_" + str(i+1)]['features_flow'] = np.load(os.path.join(path_feature_flow,f'{video_name}.npy'))
        data_summe_msva["video_" + str(i+1)]['features_rn'] = data_summe_mine["video_" + str(i+1)]['features_rn'][...]
        data_summe_msva["video_" + str(i+1)]['features_iv3'] = data_summe_mine["video_" + str(i+1)]['features_iv3'][...]
        data_summe_msva["video_" + str(i+1)]['features_3D'] = data_summe_mine["video_" + str(i+1)]['features_3D'][...]
    except:
        try:
            data_summe_msva["video_" + str(i+1)]['features_rgb'][...] = np.load(os.path.join(path_feature_rgb,f'{video_name}.npy'))
            data_summe_msva["video_" + str(i+1)]['features_flow'][...] = np.load(os.path.join(path_feature_flow,f'{video_name}.npy'))
            data_summe_msva["video_" + str(i+1)]['features_rn'][...] = data_summe_mine["video_" + str(i+1)]['features_rn'][...]
            data_summe_msva["video_" + str(i+1)]['features_iv3'][...] = data_summe_mine["video_" + str(i+1)]['features_iv3'][...]
            data_summe_msva["video_" + str(i+1)]['features_3D'][...] = data_summe_mine["video_" + str(i+1)]['features_3D'][...]
        except:
            print("Error")
#     print(sorted(list(video_mine.keys())) == sorted(list(data_summe_msva["video_" + str(i+1)].keys())))     
data_summe_msva.close()

In [161]:
data_summe_msva = h5py.File(path_summe_msva, 'r')
video_gt = data_summe_msva["video_" + str(25)]

In [162]:
print(len(list(data_summe_msva.keys())))
print(len(list(video_gt.keys())))
list(video_gt.keys())

25
15


['change_points',
 'features',
 'features_3D',
 'features_flow',
 'features_iv3',
 'features_rgb',
 'features_rn',
 'gtscore',
 'gtsummary',
 'n_frame_per_seg',
 'n_frames',
 'n_steps',
 'picks',
 'user_summary',
 'video_name']

### VSUMM - OVP

In [65]:
path_ovp_dsnet = "/data/shuaman/video_summarization/datasets/dsnet/datasets/eccv16_dataset_ovp_google_pool5.h5"
path_ovp_mine = "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_ovp_processed.h5"
data_ovp_mine = h5py.File(path_ovp_mine, 'r')
#CONCLUSION AMBOS SON LO MISMO, SOLO SIRVE QUEDARNOS CON UNO: SERA MSVA PORQUE FALTA CRUZAR CON FLOW Y RGB
data_ovp_dsnet = h5py.File(path_ovp_dsnet, 'r+')
#data_tvsum_dsnet = h5py.File(path_tvsum_dsnet, 'r')
for i in range(0,50):
    #video_gt = data_tvsum_msva["video_" + str(i+1)]
    video_mine = data_ovp_mine["video_" + str(i+1)]
    try:
    # add features
#         print(video_mine['features_rn'][...])
        data_ovp_dsnet["video_" + str(i+1)]['user_summary'] = data_ovp_mine["video_" + str(i+1)]['user_summary'][...]
        data_ovp_dsnet["video_" + str(i+1)]['video_name'] = data_ovp_mine["video_" + str(i+1)]['video_name'][...]
        data_ovp_dsnet["video_" + str(i+1)]['features_rgb'] = data_ovp_mine["video_" + str(i+1)]['features_rgb'][...]
        data_ovp_dsnet["video_" + str(i+1)]['features_flow'] = data_ovp_mine["video_" + str(i+1)]['features_flow'][...]
        data_ovp_dsnet["video_" + str(i+1)]['features_rn'] = data_ovp_mine["video_" + str(i+1)]['features_rn'][...]
        data_ovp_dsnet["video_" + str(i+1)]['features_iv3'] = data_ovp_mine["video_" + str(i+1)]['features_iv3'][...]
        data_ovp_dsnet["video_" + str(i+1)]['features_3D'] = data_ovp_mine["video_" + str(i+1)]['features_3D'][...]
    except:
        try:
            data_ovp_dsnet["video_" + str(i+1)]['user_summary'][...] = data_ovp_mine["video_" + str(i+1)]['user_summary'][...]
            data_ovp_dsnet["video_" + str(i+1)]['video_name'][...] = data_ovp_mine["video_" + str(i+1)]['video_name'][...]
            data_ovp_dsnet["video_" + str(i+1)]['features_rgb'][...] = data_ovp_mine["video_" + str(i+1)]['features_rgb'][...]
            data_ovp_dsnet["video_" + str(i+1)]['features_flow'][...] = data_ovp_mine["video_" + str(i+1)]['features_flow'][...]
            data_ovp_dsnet["video_" + str(i+1)]['features_rn'][...] = data_ovp_mine["video_" + str(i+1)]['features_rn'][...]
            data_ovp_dsnet["video_" + str(i+1)]['features_iv3'][...] = data_ovp_mine["video_" + str(i+1)]['features_iv3'][...]
            data_ovp_dsnet["video_" + str(i+1)]['features_3D'][...] = data_ovp_mine["video_" + str(i+1)]['features_3D'][...]
        except:
            print("Error")
    #print(sorted(list(video_mine.keys())) == sorted(list(data_ovp_dsnet["video_" + str(i+1)].keys())))     
data_ovp_dsnet.close()

In [77]:
data_ovp_dsnet = h5py.File(path_ovp_dsnet, 'r')
video_gt = data_ovp_dsnet["video_" + str(25)]

In [78]:
print(len(list(data_ovp_dsnet.keys())))
print(len(list(video_gt.keys())))
list(video_gt.keys())

50
14


['change_points',
 'features',
 'features_3D',
 'features_flow',
 'features_iv3',
 'features_rgb',
 'features_rn',
 'gtscore',
 'gtsummary',
 'n_frame_per_seg',
 'n_frames',
 'picks',
 'user_summary',
 'video_name']

### VSUMM - YOUTUBE

In [69]:
path_yt_dsnet = "/data/shuaman/video_summarization/datasets/dsnet/datasets/eccv16_dataset_youtube_google_pool5.h5"
path_yt_mine = "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_youtube_processed.h5"
data_yt_mine = h5py.File(path_yt_mine, 'r')
#CONCLUSION AMBOS SON LO MISMO, SOLO SIRVE QUEDARNOS CON UNO: SERA MSVA PORQUE FALTA CRUZAR CON FLOW Y RGB
data_yt_dsnet = h5py.File(path_yt_dsnet, 'r+')
#data_tvsum_dsnet = h5py.File(path_tvsum_dsnet, 'r')
for i in range(0,39):
    #video_gt = data_tvsum_msva["video_" + str(i+1)]
    if i>10:
        it = i + 12
    else:    
        it = i + 11

    video_mine = data_yt_mine["video_" + str(it)]
    try:
    # add features
#         print(video_mine['features_rn'][...])
        data_yt_dsnet["video_" + str(it)]['user_summary'] = data_yt_mine["video_" + str(it)]['user_summary'][...]
        data_yt_dsnet["video_" + str(it)]['video_name'] = data_yt_mine["video_" + str(it)]['video_name'][...]
        data_yt_dsnet["video_" + str(it)]['features_rgb'] = data_yt_mine["video_" + str(it)]['features_rgb'][...]
        data_yt_dsnet["video_" + str(it)]['features_flow'] = data_yt_mine["video_" + str(it)]['features_flow'][...]
        data_yt_dsnet["video_" + str(it)]['features_rn'] = data_yt_mine["video_" + str(it)]['features_rn'][...]
        data_yt_dsnet["video_" + str(it)]['features_iv3'] = data_yt_mine["video_" + str(it)]['features_iv3'][...]
        data_yt_dsnet["video_" + str(it)]['features_3D'] = data_yt_mine["video_" + str(it)]['features_3D'][...]
    except:
        try:
            data_yt_dsnet["video_" + str(it)]['user_summary'][...] = data_yt_mine["video_" + str(it)]['user_summary'][...]
            data_yt_dsnet["video_" + str(it)]['video_name'][...] = data_yt_mine["video_" + str(it)]['video_name'][...]
            data_yt_dsnet["video_" + str(it)]['features_rgb'][...] = data_yt_mine["video_" + str(it)]['features_rgb'][...]
            data_yt_dsnet["video_" + str(it)]['features_flow'][...] = data_yt_mine["video_" + str(it)]['features_flow'][...]
            data_yt_dsnet["video_" + str(it)]['features_rn'][...] = data_yt_mine["video_" + str(it)]['features_rn'][...]
            data_yt_dsnet["video_" + str(it)]['features_iv3'][...] = data_yt_mine["video_" + str(it)]['features_iv3'][...]
            data_yt_dsnet["video_" + str(it)]['features_3D'][...] = data_yt_mine["video_" + str(it)]['features_3D'][...]
        except:
            print("Error")
    #print(sorted(list(video_mine.keys())) == sorted(list(data_ovp_dsnet["video_" + str(i+1)].keys())))     
data_yt_dsnet.close()

In [82]:
data_yt_dsnet = h5py.File(path_yt_dsnet, 'r')
video_gt = data_yt_dsnet["video_" + str(25)]

In [83]:
print(len(list(data_yt_dsnet.keys())))
print(len(list(video_gt.keys())))
list(video_gt.keys())

39
14


['change_points',
 'features',
 'features_3D',
 'features_flow',
 'features_iv3',
 'features_rgb',
 'features_rn',
 'gtscore',
 'gtsummary',
 'n_frame_per_seg',
 'n_frames',
 'picks',
 'user_summary',
 'video_name']